In [1]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext
import pyspark.sql.functions as F

# Variables d'environnement
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--packages org.apache.hadoop:hadoop-aws:3.2.0,org.apache.hadoop:hadoop-common:3.2.0,io.trino:trino-jdbc:422 '
    'pyspark-shell'
)
os.environ['S3_ENDPOINT'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"

# On reprend la config S3 de l'autre fichier
spark = (
    SparkSession.builder
    # .master("spark://spark:7077")
    .appName("spark-silver-to-gold-1min5")
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID"))
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY"))
    .config("spark.hadoop.fs.s3a.endpoint", os.getenv("S3_ENDPOINT"))
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.attempts.maximum", "1")
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "5000")
    .config("spark.hadoop.fs.s3a.connection.timeout", "10000")
    .getOrCreate()
)

#spark.sparkContext.setLogLevel("WARN")

#Comme c'est bien foutu on peut juste lire la table en parquet comme on lisait le json
silver_path = "s3a://velib/silver/velib-disponibilite-en-temps-reel"
df_silver = spark.read.parquet(silver_path)

# Verifications
#print("Schema:")
#df_silver.printSchema()
print("\nTotal records:", df_silver.count())

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
io.trino#trino-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9f8ff537-ea0b-49e8-8508-a8f7f79716f3;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found org.apache.hadoop#hadoop-common;3.2.0 in central
	found org.apache.hadoop#hadoop-annotations;3.2.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found org.apache.httpcomponents#httpclient;4.5.2 in central
	found org.apache.httpcomponents#httpcore;4.4.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	f


Total records: 860721


In [2]:
df_silver.limit(100).toPandas().head(20)

,capacity,duedate,ebike,is_installed,is_renting,is_returning,mechanical,name,nom_arrondissement_communes,numbikesavailable,numdocksavailable,stationcode,fill_ratio,part_minute,lat,lon,part_day
0,20,2024-10-29T12:09:36+00:00,5,OUI,OUI,OUI,4,Rouget de L'isle - Watteau,Vitry-sur-Seine,9,10,44015,0.450000,2024-10-29T12:09,48.778193,2.396302,2024-10-29
1,35,2024-10-29T12:06:10+00:00,0,OUI,OUI,OUI,0,Benjamin Godard - Victor Hugo,Paris,0,35,16107,0.000000,2024-10-29T12:06,48.865983,2.275725,2024-10-29
2,25,2024-10-29T12:06:53+00:00,0,OUI,OUI,OUI,0,Cassini - Denfert-Rochereau,Paris,0,21,14111,0.000000,2024-10-29T12:06,48.837526,2.336035,2024-10-29
3,21,2024-10-29T12:11:21+00:00,6,OUI,OUI,OUI,4,Toudouze - Clauzel,Paris,10,11,9020,0.476190,2024-10-29T12:11,48.879296,2.337360,2024-10-29
4,20,2024-10-29T12:05:55+00:00,1,OUI,OUI,OUI,1,Charonne - Robert et Sonia Delaunay,Paris,2,18,11104,0.100000,2024-10-29T12:05,48.855908,2.392571,2024-10-29
5,60,2024-10-29T12:11:04+00:00,7,OUI,OUI,OUI,7,Jourdan - Stade Charléty,Paris,14,41,14014,0.233333,2024-10-29T12:11,48.819428,2.343335,2024-10-29
6,12,2024-10-29T12:11:21+00:00,7,OUI,OUI,OUI,1,Messine - Place Du Pérou,Paris,8,4,8026,0.666667,2024-10-29T12:11,48.875448,2.315508,2024-10-29
7,22,2024-10-29T12:10:19+00:00,6,OUI,OUI,OUI,1,Basilique,Saint-Denis,7,14,32017,0.318182,2024-10-29T12:10,48.936269,2.358867,2024-10-29
8,48,2024-10-29T12:07:45+00:00,0,OUI,OUI,OUI,3,Le Brun - Gobelins,Paris,3,45,13007,0.062500,2024-10-29T12:07,48.835093,2.353468,2024-10-29
9,23,2024-10-29T12:10:55+00:00,5,OUI,OUI,OUI,0,Lacépède - Monge,Paris,5,18,5110,0.217391,2024-10-29T12:10,48.843893,2.351966,2024-10-29


In [3]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Turnover (mouvements) par heure par station
turnover_window = Window.partitionBy("stationcode").orderBy("duedate")
df_gold_turnover = (
    df_silver
    .filter(~(F.col("name").isNull() | (F.lower(F.col("name")) == "None")))
    .filter(
        ~(
            F.col("name").isNull() | 
            (F.lower(F.col("name")) == "none") | 
            (F.to_date(F.substring(F.col("duedate"), 1, 10), "yyyy-MM-dd") < "2024-10-29")
        )
    )
    .withColumn("part_minute_dt", F.to_timestamp("duedate"))  # Convert string to timestamp
    .withColumn("turnover", F.abs(F.lag("numbikesavailable", 1).over(turnover_window) - F.col("numbikesavailable")))
    .withColumn("min5_interval",
    F.to_timestamp(
        F.floor(F.unix_timestamp("part_minute_dt")/300)*300
    ))
    .groupBy("stationcode", "name", "min5_interval", "lat", "lon")  # Include 'name' in groupBy
    .agg(
        F.sum("turnover").alias("total_turnover"),
    )
)
    

# Stats par heure sur l'ensemble des stations
df_gold_stats = (
    df_silver
    .filter(~(F.col("name").isNull() | (F.lower(F.col("name")) == "None")))
    .filter(
        ~(
            F.col("name").isNull() | 
            (F.lower(F.col("name")) == "none") | 
            (F.to_date(F.substring(F.col("duedate"), 1, 10), "yyyy-MM-dd") < "2024-10-29")
        )
    )
    .withColumn("part_minute_dt", F.to_timestamp("duedate"))
    .withColumn("min5_interval",
        F.to_timestamp(
            F.floor(F.unix_timestamp("part_minute_dt")/300)*300
        ))
    .groupBy("min5_interval")
    .agg(
        F.mean("numbikesavailable").alias("avg_bikes"),
        F.expr("percentile_approx(numbikesavailable, 0.5)").cast(IntegerType()).alias("median_bikes"),
        F.min("numbikesavailable").alias("min_bikes"),
        F.max("numbikesavailable").alias("max_bikes"),
        F.mean("fill_ratio").alias("avg_fill_ratio"),
        F.expr("percentile_approx(fill_ratio, 0.5)").cast(IntegerType()).alias("median_fill_ratio"),
        F.min("fill_ratio").alias("min_fill_ratio"),
        F.max("fill_ratio").alias("max_fill_ratio")
    )
    .orderBy( "min5_interval")
)

# Stats par heure pour chacune des stations
df_gold_station_stats = (
    df_silver
    .filter(~(F.col("name").isNull() | (F.lower(F.col("name")) == "None")))
    .filter(
        ~(
            F.col("name").isNull() | 
            (F.lower(F.col("name")) == "none") | 
            (F.to_date(F.substring(F.col("duedate"), 1, 10), "yyyy-MM-dd") < "2024-10-29")
        )
    )
    .withColumn("part_minute_dt", F.to_timestamp("duedate"))
    .withColumn("min5_interval",
        F.to_timestamp(
            F.floor(F.unix_timestamp("part_minute_dt")/300)*300
        ))
    .groupBy("stationcode", "name", "min5_interval", "lat", "lon")
    .agg(
        F.mean("numbikesavailable").alias("avg_bikes"),
        F.expr("percentile_approx(numbikesavailable, 0.5)").cast(IntegerType()).alias("median_bikes"),
        F.min("numbikesavailable").alias("min_bikes"),
        F.max("numbikesavailable").alias("max_bikes"),
        F.mean("fill_ratio").alias("avg_fill_ratio"),
        F.expr("percentile_approx(fill_ratio, 0.5)").cast(IntegerType()).alias("median_fill_ratio"),
        F.min("fill_ratio").alias("min_fill_ratio"),
        F.max("fill_ratio").alias("max_fill_ratio")
    )
    .orderBy("stationcode", "min5_interval")
)
#df_gold_station_stats.printSchema()

In [4]:
df_gold_stats.printSchema()
df_gold_stats.limit(20).orderBy("min5_interval").toPandas().head(20)

root
 |-- min5_interval: timestamp (nullable = true)
 |-- avg_bikes: double (nullable = true)
 |-- median_bikes: integer (nullable = true)
 |-- min_bikes: long (nullable = true)
 |-- max_bikes: long (nullable = true)
 |-- avg_fill_ratio: double (nullable = true)
 |-- median_fill_ratio: integer (nullable = true)
 |-- min_fill_ratio: double (nullable = true)
 |-- max_fill_ratio: double (nullable = true)



,min5_interval,avg_bikes,median_bikes,min_bikes,max_bikes,avg_fill_ratio,median_fill_ratio,min_fill_ratio,max_fill_ratio
0,2024-10-29 10:00:00,4.790341,2,0,46,0.168637,0,0.000000,0.964286
1,2024-10-29 10:05:00,9.755228,6,0,65,0.318137,0,0.000000,1.625000
2,2024-10-29 10:10:00,17.073877,14,0,74,0.494294,0,0.000000,1.880000
3,2024-10-29 11:00:00,4.155901,2,0,25,0.160262,0,0.000000,0.958333
4,2024-10-29 11:05:00,8.683003,5,0,63,0.292996,0,0.000000,1.575000
5,2024-10-29 11:10:00,16.652874,14,0,74,0.478000,0,0.000000,1.840000
6,2024-10-29 12:00:00,4.200883,2,0,25,0.155158,0,0.000000,0.900000
7,2024-10-29 12:05:00,8.902674,5,0,57,0.304471,0,0.000000,1.181818
8,2024-10-29 12:10:00,15.606750,12,0,74,0.448913,0,0.000000,1.761905
9,2024-10-29 12:50:00,0.740741,1,0,1,0.025543,0,0.000000,0.034483


In [5]:
df_gold_turnover.printSchema()
df_gold_turnover.limit(20).orderBy("min5_interval").toPandas().head(20)


root
 |-- stationcode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- min5_interval: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- total_turnover: long (nullable = true)



,stationcode,name,min5_interval,lat,lon,total_turnover
0,26005,Hôtel de Ville d'Asnières sur Seine,2024-10-29 10:05:00,48.909993,2.288238,30
1,30002,Jean Rostand - Paul Vaillant Couturier,2024-10-29 10:05:00,48.908168,2.453060,0
2,26005,Hôtel de Ville d'Asnières sur Seine,2024-10-29 11:05:00,48.909993,2.288238,18
3,30002,Jean Rostand - Paul Vaillant Couturier,2024-10-29 11:10:00,48.908168,2.453060,21
4,30002,Jean Rostand - Paul Vaillant Couturier,2024-10-29 12:05:00,48.908168,2.453060,12
5,26005,Hôtel de Ville d'Asnières sur Seine,2024-10-29 12:10:00,48.909993,2.288238,45
6,26005,Hôtel de Ville d'Asnières sur Seine,2024-10-29 13:05:00,48.909993,2.288238,40
7,30002,Jean Rostand - Paul Vaillant Couturier,2024-10-29 13:10:00,48.908168,2.453060,32
8,26005,Hôtel de Ville d'Asnières sur Seine,2024-10-29 14:10:00,48.909993,2.288238,19
9,26005,Hôtel de Ville d'Asnières sur Seine,2024-10-29 15:10:00,48.909993,2.288238,7


In [6]:
df_gold_station_stats.printSchema()
df_gold_station_stats.limit(20).orderBy("min5_interval").toPandas().head(20)

root
 |-- stationcode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- min5_interval: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- avg_bikes: double (nullable = true)
 |-- median_bikes: integer (nullable = true)
 |-- min_bikes: long (nullable = true)
 |-- max_bikes: long (nullable = true)
 |-- avg_fill_ratio: double (nullable = true)
 |-- median_fill_ratio: integer (nullable = true)
 |-- min_fill_ratio: double (nullable = true)
 |-- max_fill_ratio: double (nullable = true)



,stationcode,name,min5_interval,lat,lon,avg_bikes,median_bikes,min_bikes,max_bikes,avg_fill_ratio,median_fill_ratio,min_fill_ratio,max_fill_ratio
0,10003,Hauteville - Bonne Nouvelle,2024-10-29 10:10:00,48.870774,2.349650,14.275000,14,13,16,0.839706,0,0.764706,0.941176
1,10004,Metz - Faubourg Saint-Denis,2024-10-29 10:10:00,48.870893,2.353521,25.425000,25,24,28,0.529688,0,0.500000,0.583333
2,10004,Metz - Faubourg Saint-Denis,2024-10-29 11:05:00,48.870893,2.353521,29.107143,29,24,32,0.606399,0,0.500000,0.666667
3,10003,Hauteville - Bonne Nouvelle,2024-10-29 11:10:00,48.870774,2.349650,15.732143,16,14,16,0.925420,0,0.823529,0.941176
4,10003,Hauteville - Bonne Nouvelle,2024-10-29 12:05:00,48.870774,2.349650,13.250000,13,12,14,0.779412,0,0.705882,0.823529
5,10004,Metz - Faubourg Saint-Denis,2024-10-29 12:10:00,48.870893,2.353521,26.250000,26,23,31,0.546875,0,0.479167,0.645833
6,10003,Hauteville - Bonne Nouvelle,2024-10-29 13:05:00,48.870774,2.349650,11.111111,11,9,14,0.653595,0,0.529412,0.823529
7,10004,Metz - Faubourg Saint-Denis,2024-10-29 13:10:00,48.870893,2.353521,30.425926,28,26,38,0.633873,0,0.541667,0.791667
8,10003,Hauteville - Bonne Nouvelle,2024-10-29 14:05:00,48.870774,2.349650,9.000000,9,7,11,0.529412,0,0.411765,0.647059
9,10003,Hauteville - Bonne Nouvelle,2024-10-29 15:05:00,48.870774,2.349650,6.135135,6,5,7,0.360890,0,0.294118,0.411765


In [7]:
# Write the gold tables to S3
#gold_output_path = "s3a://velib/gold/"
#df_gold_turnover.write.parquet(f"{gold_output_path}turnover_rate")
#df_gold_stats.write.parquet(f"{gold_output_path}overall_station_stats")
#df_gold_station_stats.write.parquet(f"{gold_output_path}per_station_stats")

In [8]:
import trino
host, port, user = 'trino-coordinator', 8080, 'trino'
conn = trino.dbapi.connect(host=host, port=port, user=user)
cur = conn.cursor()
def createTable(df, catalog, schema_name, table, schema_location, schema, partitioned_by, external_location):
    os.environ['S3_OUTPUT_PATH'] = external_location
    #spark.sparkContext.setLogLevel("WARN")
    # Write DataFrame to S3
    (
        df.write
        .partitionBy(partitioned_by)
        .format("parquet")
        .mode("overwrite")
        .save(os.getenv('S3_OUTPUT_PATH'))
    )
    
    
    host, port, user = 'trino-coordinator', 8080, 'trino'
    conn = trino.dbapi.connect(host=host, port=port, user=user)
    cur = conn.cursor()
    
    queries = [
        f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_name} WITH (location = '{schema_location}')",
        f"DROP TABLE IF EXISTS {catalog}.{schema_name}.{table}",
        f"""
        CREATE TABLE IF NOT EXISTS {catalog}.{schema_name}.{table} (
        {schema}
        )
        WITH (
            format = 'PARQUET',
            partitioned_by = ARRAY['{partitioned_by}'],
            external_location = '{external_location}'
        )
        """,
        f"USE {catalog}.{schema_name}",
        f"CALL system.sync_partition_metadata('{schema_name}', '{table}', 'ADD')",
        # f"CALL system.sync_partition_metadata('{catalog}.{schema_name}.{table}', 'ADD')",
        f"SELECT * FROM {catalog}.{schema_name}.{table} LIMIT 5"
    ]
    
    # Execute each query in the list
    for query in queries:
        try:
            cur.execute(query)
            # Check if the query is a SELECT query to fetch results
            if query.startswith("SELECT"):vs
                results = cur.fetchall()
                for row in results:
                    print(row)
            else:
                print(f"Executed: {query}")
        except Exception as e:
            print(f"Error executing query: {query}. Error: {e}")
    # Close the cursor and connection
    cur.close()
    conn.close()

In [9]:
schema_turnover = f"""
    name VARCHAR,
    min5_interval TIMESTAMP,
    lat DOUBLE,
    lon DOUBLE,
    total_turnover BIGINT,
    stationcode VARCHAR
"""

createTable(df_gold_turnover, 'minio', 'velib_gold', 'velib_turnover_min5', 's3a://velib/gold/', schema_turnover, 'stationcode', 's3a://velib/gold/velib_turnover_min5')
 

24/10/31 12:47:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/31 12:47:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/31 12:47:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/31 12:47:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/31 12:47:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/10/31 12:47:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/10/31 12:47:32 WARN MemoryManager: Total allocation exceeds 95.

Executed: CREATE SCHEMA IF NOT EXISTS minio.velib_gold WITH (location = 's3a://velib/gold/')
Executed: DROP TABLE IF EXISTS minio.velib_gold.velib_turnover_min5
Executed: 
        CREATE TABLE IF NOT EXISTS minio.velib_gold.velib_turnover_min5 (
        
    name VARCHAR,
    min5_interval TIMESTAMP,
    lat DOUBLE,
    lon DOUBLE,
    total_turnover BIGINT,
    stationcode VARCHAR

        )
        WITH (
            format = 'PARQUET',
            partitioned_by = ARRAY['stationcode'],
            external_location = 's3a://velib/gold/velib_turnover_min5'
        )
        
Executed: USE minio.velib_gold
Executed: CALL system.sync_partition_metadata('velib_gold', 'velib_turnover_min5', 'ADD')
['Halles - Bourdonnais', datetime.datetime(2024, 10, 29, 10, 5), 48.860448992575, 2.3460162431002, 31, '1021']
['Gare du Nord - Hôpital Lariboisière', datetime.datetime(2024, 10, 29, 10, 10), 48.881949, 2.352339, 8, '10107']
['Gare du Nord - Hôpital Lariboisière', datetime.datetime(2024, 10, 29

In [11]:
schema_stats_all = f"""
    avg_bikes DOUBLE,
    median_bikes INTEGER,
    min_bikes BIGINT,
    max_bikes BIGINT,
    avg_fill_ratio DOUBLE,
    median_fill_ratio INTEGER,
    min_fill_ratio DOUBLE,
    max_fill_ratio DOUBLE,
    min5_interval TIMESTAMP
"""

createTable(df_gold_stats, 'minio', 'velib_gold', 'velib_stats_all_min5', 's3a://velib/gold/', schema_stats_all, 'min5_interval', 's3a://velib/gold/velib_stats_all_min5')

24/10/31 12:52:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/31 12:52:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/31 12:52:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/31 12:52:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/31 12:52:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/10/31 12:52:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/10/31 12:52:41 WARN MemoryManager: Total allocation exceeds 95.

Executed: CREATE SCHEMA IF NOT EXISTS minio.velib_gold WITH (location = 's3a://velib/gold/')
Executed: DROP TABLE IF EXISTS minio.velib_gold.velib_stats_all_min5
Executed: 
        CREATE TABLE IF NOT EXISTS minio.velib_gold.velib_stats_all_min5 (
        
    avg_bikes DOUBLE,
    median_bikes INTEGER,
    min_bikes BIGINT,
    max_bikes BIGINT,
    avg_fill_ratio DOUBLE,
    median_fill_ratio INTEGER,
    min_fill_ratio DOUBLE,
    max_fill_ratio DOUBLE,
    min5_interval TIMESTAMP

        )
        WITH (
            format = 'PARQUET',
            partitioned_by = ARRAY['min5_interval'],
            external_location = 's3a://velib/gold/velib_stats_all_min5'
        )
        
Executed: USE minio.velib_gold
Executed: CALL system.sync_partition_metadata('velib_gold', 'velib_stats_all_min5', 'ADD')
[12.485828551556251, 10, 0, 56, 0.39634269186717347, 0, 0.0, 1.0, datetime.datetime(2024, 10, 31, 9, 55)]
[14.032927984838913, 10, 0, 69, 0.4251062311640783, 0, 0.0, 1.76, datetime.dateti

In [7]:
schema_stats_station = f"""
    name VARCHAR,
    min5_interval TIMESTAMP,
    lat DOUBLE,
    lon DOUBLE,
    avg_bikes DOUBLE,
    median_bikes INTEGER,
    min_bikes BIGINT,
    max_bikes BIGINT,
    avg_fill_ratio DOUBLE,
    median_fill_ratio INTEGER,
    min_fill_ratio DOUBLE,
    max_fill_ratio DOUBLE,
    stationcode VARCHAR
"""

createTable(df_gold_station_stats, 'minio', 'velib_gold', 'velib_stats_station_min5', 's3a://velib/gold/', schema_stats_station, 'stationcode', 's3a://velib/gold/velib_stats_station_min5')

Executed: CREATE SCHEMA IF NOT EXISTS minio.velib_gold WITH (location = 's3a://velib/gold/')
Executed: DROP TABLE IF EXISTS minio.velib_gold.velib_stats_station_min5
Executed: 
        CREATE TABLE IF NOT EXISTS minio.velib_gold.velib_stats_station_min5 (
        
    name VARCHAR,
    min5_interval TIMESTAMP,
    lat DOUBLE,
    lon DOUBLE,
    avg_bikes DOUBLE,
    median_bikes INTEGER,
    min_bikes BIGINT,
    max_bikes BIGINT,
    avg_fill_ratio DOUBLE,
    median_fill_ratio INTEGER,
    min_fill_ratio DOUBLE,
    max_fill_ratio DOUBLE,
    stationcode VARCHAR

        )
        WITH (
            format = 'PARQUET',
            partitioned_by = ARRAY['stationcode'],
            external_location = 's3a://velib/gold/velib_stats_station_min5'
        )
        
Executed: USE minio.velib_gold
Executed: CALL system.sync_partition_metadata('velib_gold', 'velib_stats_station_min5', 'ADD')
['Petites Ecuries - Faubourg Poissonnière', datetime.datetime(2024, 10, 29, 10, 5), 48.87397217237